In [3]:
from model import *
from dilateModel import *
from data import *
from PIL import Image

In [2]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(8,'/data/spacenet/bldg/AllTrain','PAN-PNG','GT-PNG',data_gen_args,save_to_dir = None)
model = unetDil()
model_checkpoint = ModelCheckpoint('unet_membrane_AllTrain_Dil_1.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=50,epochs=5,callbacks=[model_checkpoint])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/dspuser/unet/dilateModel.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)



Epoch 1/5
Found 10586 images belonging to 1 classes.
Found 10586 images belonging to 1 classes.


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}}]]
	 [[Mean/_433]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
model = unetDil()
model.load_weights("unet_membrane_AllTrain_Dil_1.hdf5")

In [ ]:
def predImgGen(imgPath, target_size=(256,256)):
    img = io.imread(imgPath,as_gray = True)
    img = img / 255
    img = trans.resize(img,target_size)
    img = np.reshape(img,img.shape+(1,))
    img = np.reshape(img,(1,)+img.shape)
    return img

In [ ]:
def predImgGen2(imgPath, target_size=(256,256)):
    img = io.imread(imgPath,as_gray = True)
    #img = img / 255
    img = trans.resize(img,target_size)
    img = np.reshape(img,img.shape+(1,))
    img = np.reshape(img,(1,)+img.shape)
    return img

In [ ]:
test_dir = '/data/spacenet/bldg/AllTest/PAN-PNG'
for i,j,k in os.walk(test_dir):
    for file in k:
        if (os.path.splitext(file)[1] == '.png'):
            PANImgPath = os.path.join(test_dir, file)
            testImg = predImgGen(PANImgPath)
            pred = model.predict(testImg)
        
            predRe = pred.reshape(256,256)
            maskArrThresh = np.zeros((predRe.shape[0], predRe.shape[1], 4)) 
            maskArrThresh[:,:, 0] = 50
            maskArrThresh[:,:, 1] = 255
            maskArrThresh[:,:, 2] = 0
            for row in range(0, predRe.shape[0]):
                for col in range(0, predRe.shape[1]):
                    maskArrThresh[row, col, 3] = 255 - int(predRe[row, col] * 255)
            maskArrThresh = maskArrThresh.astype("uint8")
            maskImg = Image.fromarray(maskArrThresh, 'RGBA')
        
            PANImg = Image.open(PANImgPath)
            PANImgT = PANImg.convert('RGBA')
        
            PANDim = (PANImgT.width, PANImgT.height)
            maskImgRe = maskImg.resize(PANDim) 
        
            comp = Image.alpha_composite(PANImgT, maskImgRe)
            comp.save(file)